# --- Day 14: Extended Polymerization --- 

https://adventofcode.com/2021/day/14

## Get Input Data

In [2]:
def parse_data(filename):
    """Read in polymer template and pair insertion data."""
    
    with open(f'../inputs/{filename}') as file:
        lines = [line.strip() for line in file.readlines()]

    polymer_template = lines[0]

    pair_insertion_rules = {}
    for line in lines[2:]:
        # For each key, create a value that contains list of two items.
        # v[0] => The first two characters (this will get used most of the time)
        # v[1] => The full insertion string with all three characters
        # For most insertions, use v[0]; for the last insertion use v[1]
        pair_insertion_rules[line[:2]] = {
            '2_chars' : line[0] + line[-1], 
            '3_chars' : line[0] + line[-1] + line[1],
            'new_pairs' : [line[0] + line[-1], line[-1] + line[1]]
        }

    return polymer_template, pair_insertion_rules

In [3]:
test_polymer_template, test_pair_insertion_rules = parse_data('test_polymer_data.txt')
test_polymer_template, test_pair_insertion_rules

('NNCB',
 {'CH': {'2_chars': 'CB', '3_chars': 'CBH', 'new_pairs': ['CB', 'BH']},
  'HH': {'2_chars': 'HN', '3_chars': 'HNH', 'new_pairs': ['HN', 'NH']},
  'CB': {'2_chars': 'CH', '3_chars': 'CHB', 'new_pairs': ['CH', 'HB']},
  'NH': {'2_chars': 'NC', '3_chars': 'NCH', 'new_pairs': ['NC', 'CH']},
  'HB': {'2_chars': 'HC', '3_chars': 'HCB', 'new_pairs': ['HC', 'CB']},
  'HC': {'2_chars': 'HB', '3_chars': 'HBC', 'new_pairs': ['HB', 'BC']},
  'HN': {'2_chars': 'HC', '3_chars': 'HCN', 'new_pairs': ['HC', 'CN']},
  'NN': {'2_chars': 'NC', '3_chars': 'NCN', 'new_pairs': ['NC', 'CN']},
  'BH': {'2_chars': 'BH', '3_chars': 'BHH', 'new_pairs': ['BH', 'HH']},
  'NC': {'2_chars': 'NB', '3_chars': 'NBC', 'new_pairs': ['NB', 'BC']},
  'NB': {'2_chars': 'NB', '3_chars': 'NBB', 'new_pairs': ['NB', 'BB']},
  'BN': {'2_chars': 'BB', '3_chars': 'BBN', 'new_pairs': ['BB', 'BN']},
  'BB': {'2_chars': 'BN', '3_chars': 'BNB', 'new_pairs': ['BN', 'NB']},
  'BC': {'2_chars': 'BB', '3_chars': 'BBC', 'new_pairs'

In [4]:
polymer_template, pair_insertion_rules = parse_data('polymer_data.txt')

## Part 1
---

In [5]:
from more_itertools import pairwise
from collections import Counter

In [6]:
def calc_part_1(template, rules, num_steps):
    """Return the difference of the most and least common counts of a polymer template, 
    after pair insertion rules have been applied for num_steps.
    """

    for _ in range(num_steps):
        pairs = list(pairwise(template))

        next_template = ''
        for i, pair in enumerate(pairs):
            
            # Most insertions, append just the first two characters
            if i != len(pairs) - 1:
                next_template += ''.join(rules[''.join(pair)]['2_chars'])
            
            # For the last last insertion, insertion, append all three
            else:
                next_template += ''.join(rules[''.join(pair)]['3_chars'])

        template = next_template

    counts = Counter(template)

    most_common = counts.most_common()[0][1]
    least_common = counts.most_common()[-1][1]
    diff = most_common - least_common

    return diff

### Run on Test Data

In [7]:
calc_part_1(test_polymer_template, test_pair_insertion_rules, 10)  # Should return 1588

1588

### Run on Input Data

In [8]:
calc_part_1(polymer_template, pair_insertion_rules, 10)

3118

## Part 2
---

Part 2 is another "Your answer to Part 1 was probably too naive/inefficient." type of problem.

Gotta run this one out 40 steps (instead of 10).

I'm thinking I need to just keep track of the counts, and **NOT** try to build a new template at each step.

In [ ]:
def calc_part_2(template, rules, num_steps):
    """Return the difference of the most and least common counts of a polymer template, 
    after pair insertion rules have been applied for num_steps.
    """

    first_char = template[0]

    pairs = [''.join(p) for p in pairwise(template)]
    pairs_counts = Counter(pairs)


    for i in range(num_steps):
        print(f'\nstep {i+1}')
        print(f'beginning pairs counts: {pairs_counts}')

        foo = list(pairs_counts)  # old
        bar = []                  # new
        
        old_pairs = Counter(list(pairs_counts))  # This makes a copy of pairs_counts?
        # new_pairs = Counter()
        new_pairs = []   ################  THIS IS BLOWING UP!!!!! I CAN'T USE THIS.
        for pair in list(pairs_counts):
            # count_of_pair = pairs_counts[pair]
            # print(pair, count_of_pair)

            # Add new pairs
            # for p in rules[pair]['new_pairs']:
            #     # print(f'pair: {pair}, p: {p}, pairs_counts[pair]: {pairs_counts[pair]}')
            #     # pairs_counts[p] += pairs_counts[pair]
            #     new_pairs[p] += pairs_counts[pair]


            # Subtract old pairs
            # pairs_counts[pair] -= count_of_pair

            # bar += rules[pair]['new_pairs'] * pairs_counts[pair]
            new_pairs += rules[pair]['new_pairs'] * pairs_counts[pair]

            # # if pair == first_pair:
            # #     first_pair = new_pairs[0]

            # if i == 2: print(f'old pair: {pair}  --> new pairs: {new_pairs}')
            # pairs_counts[pair] -= 1  # Reset to 0?
            # pairs_counts += Counter(new_pairs)   # Then add in the new pairs created in its place
            # if i == 2: print(f'pair counts after adding new pairs: {pairs_counts}')

        # print(f'step {i+1}\nlen of old pairs: {len(old_pairs)}\nlen new pairs: {len(new_pairs)}')
        # # Subtract old pairs
        for pair in old_pairs:
        #     # if i == 2: print(f'old pair: {pair} --> new pairs: {new_pairs}')
            pairs_counts[pair] -= pairs_counts[pair]

        # # # print(f'Old counts to remove: {Counter(old_pairs)}')
        # # # print(f'new pairs to add: {Counter(new_pairs)}')
        # Then add new pairs
        pairs_counts += Counter(new_pairs)

        # print(f'Old counts to remove: {old_pairs}')
        # print(f'new pairs to add: {new_pairs}')

        # pairs_counts = pairs_counts - old_pairs + new_pairs

        print(f'ending pairs counts: {pairs_counts}')

    single_char_counts = Counter()
    for j, p in enumerate(pairs_counts):

        # if p == first_pair:
        #     if i == 2: print(f'first pair {first_pair}')
        #     single_char_counts[p[0]] += 1
        #     single_char_counts[p[1]] += pairs_counts[p]
        #     if i == 2: print(f'single char counts, j=={j}: {single_char_counts}')
        # else:
            single_char_counts[p[1]] += pairs_counts[p]
            # if i == 2: print(f'single char counts, j=={j}: {p} getting added to {single_char_counts}')

    # Add in a count for the first character in the template
    single_char_counts[first_char] += 1

        # print(f'Single char counts: {single_char_counts}')

    most_common = single_char_counts.most_common()[0][1]
    least_common = single_char_counts.most_common()[-1][1]
    diff = most_common - least_common

    return diff

In [173]:
def calc_part_2(template, rules, num_steps):
    """Return the difference of the most and least common counts of a polymer template, 
    after pair insertion rules have been applied for num_steps.
    """

    first_char = template[0]

    pairs = [''.join(p) for p in pairwise(template)]
    pairs_counts = Counter(pairs)

    for i in range(num_steps):

        print(f'\nstep {i+1}')
        print(f'beginning pairs counts: {pairs_counts}')

        old_pairs = list(pairs_counts)
        new_pairs = Counter()
        for pair in list(pairs_counts):
            # Add new pairs
            for p in rules[pair]['new_pairs']:
                new_pairs[p] += pairs_counts[pair]

        pairs_counts = pairs_counts - Counter(old_pairs) + new_pairs
        print(f'ending pairs counts: {pairs_counts}')


    single_char_counts = Counter()
    for p in enumerate(pairs_counts):
        single_char_counts[p[1]] += pairs_counts[p]

    # Add in a count for the first character in the template
    single_char_counts[first_char] += 1

    most_common = single_char_counts.most_common()[0][1]
    least_common = single_char_counts.most_common()[-1][1]
    diff = most_common - least_common

    return diff

In [174]:
calc_part_2(test_polymer_template, test_pair_insertion_rules, 3)  # Should return 1588, just like in part 1


step 1
beginning pairs counts: Counter({'NN': 1, 'NC': 1, 'CB': 1})
ending pairs counts: Counter({'NC': 1, 'CN': 1, 'NB': 1, 'BC': 1, 'CH': 1, 'HB': 1})

step 2
beginning pairs counts: Counter({'NC': 1, 'CN': 1, 'NB': 1, 'BC': 1, 'CH': 1, 'HB': 1})
ending pairs counts: Counter({'NB': 2, 'BC': 2, 'BB': 2, 'CB': 2, 'CC': 1, 'CN': 1, 'BH': 1, 'HC': 1})

step 3
beginning pairs counts: Counter({'NB': 2, 'BC': 2, 'BB': 2, 'CB': 2, 'CC': 1, 'CN': 1, 'BH': 1, 'HC': 1})
ending pairs counts: Counter({'NB': 5, 'BB': 5, 'BC': 4, 'HB': 3, 'CN': 2, 'BN': 2, 'CH': 2, 'CB': 1, 'NC': 1, 'CC': 1, 'BH': 1, 'HH': 1})


1

In [ ]:
ending pairs counts: Counter({'NB': 4, 'BB': 4, 'BC': 3, 'HB': 3, 'CN': 2, 'BN': 2, 'CH': 2, 'CC': 1, 'BH': 1, 'NC': 1, 'HH': 1})

### Run on Test Data

In [159]:
calc_part_2(test_polymer_template, test_pair_insertion_rules, 3)  # Should return 1588, just like in part 1


step 1
beginning pairs counts: Counter({'NN': 1, 'NC': 1, 'CB': 1})
ending pairs counts: Counter({'NC': 1, 'CN': 1, 'NB': 1, 'BC': 1, 'CH': 1, 'HB': 1})

step 2
beginning pairs counts: Counter({'NC': 1, 'CN': 1, 'NB': 1, 'BC': 1, 'CH': 1, 'HB': 1})
ending pairs counts: Counter({'NB': 2, 'BC': 2, 'BB': 2, 'CB': 2, 'CN': 1, 'CC': 1, 'BH': 1, 'HC': 1})

step 3
beginning pairs counts: Counter({'NB': 2, 'BC': 2, 'BB': 2, 'CB': 2, 'CN': 1, 'CC': 1, 'BH': 1, 'HC': 1})
ending pairs counts: Counter({'NB': 4, 'BB': 4, 'BC': 3, 'HB': 3, 'CN': 2, 'BN': 2, 'CH': 2, 'CC': 1, 'BH': 1, 'NC': 1, 'HH': 1})

step 4
beginning pairs counts: Counter({'NB': 4, 'BB': 4, 'BC': 3, 'HB': 3, 'CN': 2, 'BN': 2, 'CH': 2, 'CC': 1, 'BH': 1, 'NC': 1, 'HH': 1})
ending pairs counts: Counter({'NB': 9, 'BB': 9, 'BN': 6, 'CB': 5, 'BC': 4, 'CN': 3, 'BH': 3, 'HC': 3, 'CC': 2, 'NC': 1, 'HH': 1, 'HN': 1, 'NH': 1})

step 5
beginning pairs counts: Counter({'NB': 9, 'BB': 9, 'BN': 6, 'CB': 5, 'BC': 4, 'CN': 3, 'BH': 3, 'HC': 3, '

1588

In [169]:
calc_part_2(test_polymer_template, test_pair_insertion_rules, 40)  # Should return 2188189693529


step 1
beginning pairs counts: Counter({'NN': 1, 'NC': 1, 'CB': 1})
ending pairs counts: Counter({'NC': 2, 'NN': 1, 'CB': 1, 'CN': 1, 'NB': 1, 'BC': 1, 'CH': 1, 'HB': 1})

step 2
beginning pairs counts: Counter({'NC': 2, 'NN': 1, 'CB': 1, 'CN': 1, 'NB': 1, 'BC': 1, 'CH': 1, 'HB': 1})
ending pairs counts: Counter({'NC': 3, 'CB': 2, 'CN': 2, 'NB': 2, 'BC': 2, 'CH': 2, 'HB': 2, 'NN': 1, 'CC': 1, 'BB': 1, 'BH': 1, 'HC': 1})

step 3
beginning pairs counts: Counter({'NC': 3, 'CB': 2, 'CN': 2, 'NB': 2, 'BC': 2, 'CH': 2, 'HB': 2, 'NN': 1, 'CC': 1, 'BB': 1, 'BH': 1, 'HC': 1})
ending pairs counts: Counter({'NC': 4, 'CB': 3, 'CN': 3, 'NB': 3, 'BC': 3, 'CH': 3, 'HB': 3, 'CC': 2, 'BB': 2, 'BH': 2, 'HC': 2, 'NN': 1, 'BN': 1, 'HH': 1})

step 4
beginning pairs counts: Counter({'NC': 4, 'CB': 3, 'CN': 3, 'NB': 3, 'BC': 3, 'CH': 3, 'HB': 3, 'CC': 2, 'BB': 2, 'BH': 2, 'HC': 2, 'NN': 1, 'BN': 1, 'HH': 1})
ending pairs counts: Counter({'NC': 5, 'CB': 4, 'CN': 4, 'NB': 4, 'BC': 4, 'CH': 4, 'HB': 4, 'CC': 3

1

In [101]:
calc_part_2(test_polymer_template, test_pair_insertion_rules, 3)  # Should return 2188189693529


step 1
beginning pairs counts: Counter({'NN': 1, 'NC': 1, 'CB': 1})
Old counts to remove: Counter({'NN': 1, 'NC': 1, 'CB': 1})
new pairs to add: Counter({'NC': 1, 'CN': 1, 'NB': 1, 'BC': 1, 'CH': 1, 'HB': 1})
ending pairs counts: Counter({'NC': 1, 'CN': 1, 'NB': 1, 'BC': 1, 'CH': 1, 'HB': 1})
Single char counts: Counter({'C': 2, 'N': 2, 'B': 2, 'H': 1})

step 2
beginning pairs counts: Counter({'NC': 1, 'CN': 1, 'NB': 1, 'BC': 1, 'CH': 1, 'HB': 1})
Old counts to remove: Counter({'NC': 1, 'CN': 1, 'NB': 1, 'BC': 1, 'CH': 1, 'HB': 1})
new pairs to add: Counter({'NB': 2, 'BC': 2, 'BB': 2, 'CB': 2, 'CC': 1, 'CN': 1, 'BH': 1, 'HC': 1})
ending pairs counts: Counter({'NB': 2, 'BC': 2, 'BB': 2, 'CB': 2, 'CN': 1, 'CC': 1, 'BH': 1, 'HC': 1})
Single char counts: Counter({'B': 6, 'C': 4, 'N': 2, 'H': 1})

step 3
beginning pairs counts: Counter({'NB': 2, 'BC': 2, 'BB': 2, 'CB': 2, 'CN': 1, 'CC': 1, 'BH': 1, 'HC': 1})
old pair: CN  --> new pairs: ['CC', 'CN']
old pair: NB  --> new pairs: ['CC', 'CN'

7

In [21]:
Counter('NCNBCHB')

Counter({'N': 2, 'C': 2, 'B': 2, 'H': 1})

In [22]:
Counter('NBCCNBBBCBHCB')

Counter({'N': 2, 'B': 6, 'C': 4, 'H': 1})

In [97]:
Counter([''.join(p) for p in pairwise('NBCCNBBBCBHCB')])

Counter({'NB': 2,
         'BC': 2,
         'CC': 1,
         'CN': 1,
         'BB': 2,
         'CB': 2,
         'BH': 1,
         'HC': 1})

In [ ]:
ending pairs counts: Counter({'NB': 2, 'BC': 2, 'BB': 2, 'CB': 2, 'CN': 1, 'CC': 1, 'BH': 1, 'HC': 1})

In [64]:
Counter('NBBBCNCCNBBNBNBBCHBHHBCHB')

Counter({'N': 5, 'B': 11, 'C': 5, 'H': 4})

In [69]:
Counter([''.join(p) for p in pairwise('NBBBCNCCNBBNBNBBCHBHHBCHB')])

Counter({'NB': 4,
         'BB': 4,
         'BC': 3,
         'CN': 2,
         'NC': 1,
         'CC': 1,
         'BN': 2,
         'CH': 2,
         'HB': 3,
         'BH': 1,
         'HH': 1})

In [ ]:
ending pairs counts: Counter({'NB': 3, 'BC': 3, 'BB': 3, 'CN': 2, 'HB': 2, 'CC': 1, 'CB': 1, 'BH': 1, 'NC': 1, 'BN': 1, 'CH': 1, 'HH': 1})

In [74]:
Counter([''.join(p) for p in pairwise('NBCCNBBBCBHCB')])

Counter({'NB': 2,
         'BC': 2,
         'CC': 1,
         'CN': 1,
         'BB': 2,
         'CB': 2,
         'BH': 1,
         'HC': 1})

### Run on Input Data